In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from src.models import unet_generator
from src.generate_image import generate_image, GAN_image_folder
import os, shutil
from sys import argv
from src.make_dataset import parse_and_save_dir

In [2]:
gaussian   = 0.1 

identifier = "20240426_DATA_brightfield_mixed_edit"


parent_dir = "/mnt/d/cycle_gan/"
exp_dir_raw = parent_dir + "data/exp/brightfield_mixed_edit_real/"
sim_dir_raw = parent_dir + "data/sim/brightfield_mixed_synth/"

exp_dir = exp_dir_raw[:-1] + "_256_slices/"
sim_dir = sim_dir_raw[:-1] + "_gauss_{}_256_slices/".format(gaussian)

In [3]:
checkpoint_path = parent_dir + "cycle_gan_results/checkpoints/checkpoint_{}".format(identifier)

num_channels = 1
generator_exp = unet_generator(num_channels, 1, "instancenorm")
generator_sim = unet_generator(1, num_channels, "instancenorm")

# ## Checkpoints
checkpoint_path = parent_dir + "cycle_gan_results/checkpoints/checkpoint_{}".format(identifier)

ckpt = tf.train.Checkpoint(generator_exp=generator_exp,
                           generator_sim=generator_sim)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=None)

# if a checkpoint exists, restore the latest checkpoint.
num_ckpts = len(ckpt_manager.checkpoints)
print(" number of checkpoints ", num_ckpts)

def cycle(x):
    return generator_sim(generator_exp(x))

 number of checkpoints  41


In [4]:
sim_dir = sim_dir_raw[:-1] + "_gauss_{}_256_slices_TO_GAN/".format(gaussian)
parse_and_save_dir(sim_dir_raw, sim_dir, 256, 256, num_channels=num_channels, gaussian=gaussian)

In [5]:
#GAN A FOLDER
for i in range(0, num_ckpts):
    ckpt.restore(ckpt_manager.checkpoints[i])
    print("Restored from {}".format(ckpt_manager.checkpoints[i]))
    print("Cycle GAN")

    model      = generator_exp
    data_dir   = sim_dir
    ganned_dir = parent_dir + "data/GAN/exp/{}_CKPT_{}/".format(identifier, i)
    GAN_image_folder(model, data_dir, ganned_dir, 256, 256, False)

Restored from /mnt/d/cycle_gan/cycle_gan_results/checkpoints/checkpoint_20240426_DATA_brightfield_mixed_edit/ckpt-1
Cycle GAN
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading model
stride: 256
predicting data
stitching data
saving data
processing data
loading 

In [ ]:
#GAN A FOLDER
i=40
ckpt.restore(ckpt_manager.checkpoints[i])
print("Restored from {}".format(ckpt_manager.checkpoints[i]))
print("Cycle GAN")

model      = generator_exp
data_dir   = sim_dir
ganned_dir = parent_dir + "data/GAN/exp/RAW_{}_CKPT_{}/".format(identifier, i)
GAN_image_folder(model, data_dir, ganned_dir, 256, 256, False)

In [ ]:
model = generator_exp
ckpt.restore(ckpt_manager.checkpoints[34])
input_file = parent_dir + "data/sim/raw/chels/chels.tif"
generate_image(model, input_file, 256, stride=256, avg=False, \
        plot=False, save_data=True, save_dir=parent_dir,fname='data/sim/raw/chels/generated_chels.tif')

In [ ]:
model = generator_exp
sim_dir_raw = parent_dir + "data/denser/masks/"
data_dir = sim_dir_raw[:-1] + "_gauss_{}_256_slices_small/".format(gaussian)
ganned_dir = data_dir[:-1] + "_GANNED/"
parse_and_save_dir(sim_dir_raw, data_dir, 256, 256, num_channels=1, gaussian=gaussian)
#GAN_image_folder(model, data_dir, ganned_dir, 256, 256, False)